In [12]:
import watertap.examples.flowsheets.electrodialysis.analysis.Analyze_ED_1stack as ef
import importlib
from idaes.core.solvers import get_solver
import pandas as pd
import numpy as np
from pyomo.environ import value, check_optimal_termination
import idaes.logger as idaeslog
from idaes.core.util.exceptions import  InitializationError
from idaes.core.util.model_statistics import report_statistics, degrees_of_freedom, fixed_variables_generator
importlib.reload(ef)

<module 'watertap.examples.flowsheets.electrodialysis.analysis.Analyze_ED_1stack' from '/Users/xb/Documents/Projects/watertap/watertap/examples/flowsheets/electrodialysis/analysis/Analyze_ED_1stack.py'>

In [24]:
m = ef.build()
# set initial feed condition 
initarg = ef.make_initarg_list([15])
deci_var_dict = {
    "voltage_applied[0]": 10,
    "recovery_vol_H2O": 0.9,
}
ef.initialize_dof0_system(
    m=m, initargs=initarg[0], solve_after_init=True, **deci_var_dict
)
print("===INITIALIZE OUTCOME")
ef.display_model_metrics(m)
opt_var_dict = {
    "voltage_applied[0]": (10, 0, value(m.fs.voltage_lim)),
    "cell_pair_num": (50, 1, 10000),
    "cell_length": (1.68, 0.01, 10),
    "cell_width": (0.197, 0.01, 10),
    "channel_height": (7e-4, 1e-4, 1e-2)
}
if not m.find_component("obj") is None:
    m.del_component("obj")
    print("obj==0 deleted")
opt_res=ef.optimize_LCOW_fixed_prod_salinity(m, prod_sal=0.5, **opt_var_dict)
print("===OPTIMIZE OUTCOME===")
ef.display_model_metrics(m)
print(f"Final opt solver condition is {opt_res.solver.termination_condition}.")

2023-05-03 12:42:17 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-05-03 12:42:17 [WARNING] idaes.core.base.costing_base: flow_expr is an expression with a lower bound of less than zero. Costing requires that all flows have a lower bound equal to or greater than zero to avoid negative costs.
2023-05-03 12:42:17 [WARNING] idaes.watertap.property_models.multicomp_aq_sol_prop_pack: While using the calculate_state method on fs.feed.properties[0.0], variable flow_vol_phase was provided as an argument in var_args, but it does not have a scaling factor. This suggests that the calculate_scaling_factor method has not been used or the variable was created on demand after the scaling factors were calculated. It is recommended to touch all relevant variables (i.e. call them or set an initial value) before using the calculate_scalin

===INITIALIZE OUTCOME
---Flow properties in feed, product and disposal---
                                          Feed ED_diluate_in  \
Volume Flow Rate (m3/s)                0.00052      0.000476   
Total Ion Mass Concentration (kg/m3)  15.00000                 

                                     ED_concentrate_in    Product   Disposal  
Volume Flow Rate (m3/s)                       0.000476   0.000468   0.000052  
Total Ion Mass Concentration (kg/m3)                    12.983222  33.151000  
---Performance Metrics---
                                                      value
Water recovery by volume                           0.900000
Total membrane area (aem or cem), m2              33.096000
ED cell pair number                              100.000000
ED channel height, m                               0.000710
ED cell flow path length                           1.680000
ED cell width                                      0.197000
Operation Voltage, V                              